In [1]:
from scipy import stats
import numpy as np
import pandas as pd
import torch
import blockers
import pruners
import torch_subspace
from experiments.models import get_model
from experiments.data import get_data

# Load model
We compute different pruning scores on the same model with the same blocking scheme

In [2]:
device = torch.device("cuda:7")
train_data, test_data, num_classes = get_data(
    "cifar10", batch_size=256, data_path="./data"
)
model = get_model("vgg16", num_classes=num_classes, device=device)
model.load_state_dict(torch.load("./checkpoint/vgg16_cifar10_epoch_160"))
torch_subspace.convert_model_to_lr(model)
blockers.square.make_blocks(model)


# Pruning score
We calculate scores for two different pruning methods

In [3]:
scores = {}
print("Calculating rel error scores")
scores["rel_error"] = pruners.rel_error._compute_scores(model=model)
print("Calculating magnitude scores")
scores["magnitude"] = pruners.magnitude._compute_scores(model=model)
print("Calculating alignment output scores")
scores["alignment_output"] = pruners.alignment_output._compute_scores(model=model, train_data=train_data, device=device)
print("Calculating alignment output sampling scores")
scores["alignment_output_sampling"] = pruners.alignment_output_sampling._compute_scores(model=model, train_data=train_data, sparsity=0.95, proportional_sampling=False, device=device)
print("Calculating alignment variance scores")
scores["alignment_variance"] = pruners.alignment_variance._compute_scores(model, train_data, device)

Calculating rel error scores
Scoring module: 146 / 146
Calculating magnitude scores
Scoring module: 146 / 146
Calculating alignment output scores
Scoring module 146 / 146 (mask    9 /   10)
Calculating alignment output sampling scores
Iter: 999
Calculating alignment variance scores
Scoring module 146 / 146 (mask    9 /   10)


# Evaluate correlations
First we look at block by block correlation in scores

In [4]:
scores_flat = {}
for key, val in scores.items():
    scores_flat[key] = np.concatenate(val)
scores_flat = pd.DataFrame(scores_flat)
scores_flat.corr(method="spearman")

,rel_error,magnitude,alignment_output,alignment_output_sampling,alignment_variance
rel_error,1.000000,0.906422,0.872102,0.008642,0.303540
magnitude,0.906422,1.000000,0.899813,0.011311,0.277830
alignment_output,0.872102,0.899813,1.000000,0.008240,0.290989
alignment_output_sampling,0.008642,0.011311,0.008240,1.000000,0.051718
alignment_variance,0.303540,0.277830,0.290989,0.051718,1.000000


In [6]:
for i in range(len(scores["rel_error"])):
    display(stats.spearmanr(scores["rel_error"][i], scores["alignment_variance"][i]))

SpearmanrResult(correlation=0.5441360298008848, pvalue=0.0033454791078566955)

SpearmanrResult(correlation=0.5875879087686505, pvalue=0.0012699092296497645)

SpearmanrResult(correlation=0.313646563942557, pvalue=0.011610939343114749)

SpearmanrResult(correlation=0.41593113866797005, pvalue=0.000630482071238406)

SpearmanrResult(correlation=0.566877717681815, pvalue=1.0427563360972673e-06)

...

SpearmanrResult(correlation=0.5276492843761433, pvalue=0.11699832200012014)

SpearmanrResult(correlation=0.4697700535929367, pvalue=0.1707074545917416)

SpearmanrResult(correlation=0.5276492843761433, pvalue=0.11699832200012014)

SpearmanrResult(correlation=0.6227991553292185, pvalue=0.054438696140665634)

SpearmanrResult(correlation=-0.06919990614769093, pvalue=0.8493480024065823)